In [4]:
import pandas as pd
from pulp import *

In [5]:
n = 5
cities = [1,2,3,4,5]
dist = pd.read_csv('data_tsp.csv',header=None)	

In [6]:
dist.head()

,0,1,2,3,4
0,1234567,10.0,8.0,9.0,7.0
1,10,1234567.0,10.0,5.0,6.0
2,8,10.0,1234567.0,8.0,9.0
3,9,5.0,8.0,1234567.0,6.0
4,7,6.0,9.0,6.0,1234567.0


In [7]:
x = LpVariable.dicts('X', [(c1, c2) for c1 in cities for c2 in cities], cat='Binary')  # Creating the pairwise binary variable iterate over the cities list twice.


In [8]:
x

{(1, 1): X_(1,_1),
 (1, 2): X_(1,_2),
 (1, 3): X_(1,_3),
 (1, 4): X_(1,_4),
 (1, 5): X_(1,_5),
 (2, 1): X_(2,_1),
 (2, 2): X_(2,_2),
 (2, 3): X_(2,_3),
 (2, 4): X_(2,_4),
 (2, 5): X_(2,_5),
 (3, 1): X_(3,_1),
 (3, 2): X_(3,_2),
 (3, 3): X_(3,_3),
 (3, 4): X_(3,_4),
 (3, 5): X_(3,_5),
 (4, 1): X_(4,_1),
 (4, 2): X_(4,_2),
 (4, 3): X_(4,_3),
 (4, 4): X_(4,_4),
 (4, 5): X_(4,_5),
 (5, 1): X_(5,_1),
 (5, 2): X_(5,_2),
 (5, 3): X_(5,_3),
 (5, 4): X_(5,_4),
 (5, 5): X_(5,_5)}

In [9]:
u = LpVariable.dicts('U', [c1 for c1 in cities], lowBound=0, upBound=(n-1), cat='Integer') # Create a dictionary 'u' holding an integer LpVariable for each city. Hence iterate over the list of cities only once.


In [10]:
u

{1: U_1, 2: U_2, 3: U_3, 4: U_4, 5: U_5}

In [11]:
model = LpProblem("TSP", LpMinimize)
model += lpSum([dist.iloc[c1-1, c2-1] * x[(c1, c2)] for c1 in cities for c2 in cities]) # Summing together, the distance between each pair 

In [12]:
for c2 in cities:
    model += lpSum([x[(c1, c2)] for c1 in cities]) == 1 # To ensure that a city can be arrived from excatly one other city 
for c1 in cities:
    model += lpSum([x[(c1, c2)] for c2 in cities]) == 1 # To ensure that ther can be a departure to excatly one other city from a city


In [13]:
for c2 in cities:
    for c1 in cities:
        model += x[(c2,c1)]+x[(c1,c2)]<=1 # To eliminate subtours of length two  


In [14]:
status = model.solve()
print(LpStatus[status])


Optimal


In [15]:
# Print the distance travelled in optimal solution
print(value(model.objective)) 

# Print the route travelled 
for c2 in cities:
    for c1 in cities:
        if x[(c1,c2)].varValue ==1:
            print(c1," -> ",c2)

34.0
5  ->  1
4  ->  2
1  ->  3
3  ->  4
2  ->  5
